In [1]:
#MRI Brain Tumor Dataset

link: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection

In [2]:
#import packages

In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import random
import cv2
import sys
import torch.nn as nn
import torch.nn.functional as F
import seaborn as sns

In [4]:
#Reading MRI Images

In [5]:
malignant = []
benign = []
for f in glob.iglob("./brain_tumor_dataset/yes/*.[jJ][pP][gG]"):
    img = cv2.imread(f)
    img = cv2.resize(img,(128,128))
    b, g, r = cv2.split(img)
    img = cv2.merge([r,g,b])
    malignant.append(img)

for f in glob.iglob("./brain_tumor_dataset/no/*.[jJ][pP][gG]"):
    img = cv2.imread(f)
    img = cv2.resize(img,(128,128)) 
    b, g, r = cv2.split(img)
    img = cv2.merge([r,g,b])
    benign.append(img)

In [6]:
malignant = np.array(malignant)
benign = np.array(benign)
combined = np.concatenate((benign, malignant))

In [7]:
malignant.shape

(154, 128, 128, 3)

In [8]:
benign.shape

(91, 128, 128, 3)

In [9]:
np.random.choice(10, 5, replace=False)

array([9, 7, 4, 3, 8])

In [10]:
#Visualizing MRI Images (Images shown below are not trained, so they are not accurate)

In [11]:
def plot_all(benign, malignant, cols=5):
    def plot_category(images, title):
        num_images = len(images)
        rows = (num_images + cols - 1) // cols

        plt.figure(figsize=(16, 2 * rows)) 
        for i, img in enumerate(images, 1):
            plt.subplot(rows, cols, i)
            plt.imshow(img)
            plt.title(title)
            plt.axis('off')
        plt.tight_layout()
        plt.show()

    # Plot benign images
    plot_category(benign, "Safe")

    # Plot malignant images
    plot_category(malignant, "Tumor")

In [ ]:
plot_all(benign, malignant)

In [ ]:
#Pytorch's Abstract Dataset Class

In [ ]:
class Dataset(object):
    """An abstract class representing a Dataset.

    All other datasets should subclass it. All subclasses should override
    ``__len__``, that provides the size of the dataset, and ``__getitem__``,
    supporting integer indexing in range from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

In [ ]:
IMAGE_SIZE = (128, 128)

def load_and_process_image(file_path):
    """
    Load and preprocess an image.
    """
    try:
        img = cv2.imread(file_path)
        img = cv2.resize(img, IMAGE_SIZE)
        b, g, r = cv2.split(img)
        img = cv2.merge([r, g, b])
        img = img.reshape((img.shape[2], img.shape[0], img.shape[1]))
        return img
    except Exception as e:
        print(f"Error loading image {file_path}: {e}")
        return None

class MRI(Dataset):
    """
    Custom Dataset class for MRI images.
    """
    def __init__(self):
        tumor = [load_and_process_image(f) for f in glob.iglob("./data/brain_tumor_dataset/yes/*.[jJ][pP][gG]")]
        safe = [load_and_process_image(f) for f in glob.iglob("./data/brain_tumor_dataset/no/*.[jJ][pP][gG]")]
        
        tumor = np.array([img for img in tumor if img is not None], dtype=np.float32)
        safe = np.array([img for img in safe if img is not None], dtype=np.float32)
        
        tumor_label = np.ones(tumor.shape[0], dtype=np.float32)
        safe_label = np.zeros(safe.shape[0], dtype=np.float32)
        
        self.images = np.concatenate((tumor, safe), axis=0)
        self.labels = np.concatenate((tumor_label, safe_label))

    def __len__(self):
        return self.images.shape[0]
    
    def __getitem__(self, index):
        sample = {'image': self.images[index], 'label': self.labels[index]}
        return sample
    
    def normalize(self):
        self.images = self.images / 255.0

In [ ]:
mri_dataset = MRI()
mri_dataset.normalize()

In [ ]:
#dataloader

In [ ]:
names = {0: 'Safe', 1: 'Tumor'}
dataloader = DataLoader(mri_dataset, shuffle=True)

for i, sample in enumerate(dataloader):
    img = sample['image'].squeeze()
    img = img.reshape((img.shape[1], img.shape[2], img.shape[0]))
    plt.title(names[sample['label'].item()])
    plt.imshow(img)
    plt.show()

In [ ]:
#creating model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn_model = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=5),
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=5))
        
        self.fc_model = nn.Sequential(
        nn.Linear(in_features=256, out_features=120),
        nn.Tanh(),
        nn.Linear(in_features=120, out_features=84),
        nn.Tanh(),
        nn.Linear(in_features=84, out_features=1))
        
    def forward(self, x):
        x = self.cnn_model(x)
        x = x.view(x.size(0), -1)
        x = self.fc_model(x)
        x = F.sigmoid(x)
        
        return x
            

In [ ]:
model = CNN()

In [ ]:
model

In [ ]:
model.cnn_model[0].weight

In [ ]:
model.cnn_model[0].weight.shape

In [ ]:
model.cnn_model[0].weight[0][1]

In [ ]:
#Linear Layer

In [ ]:
model.fc_model

In [ ]:
model.fc_model[0]

In [ ]:
model.fc_model[0].weight

In [ ]:
model.fc_model[0].weight.shape

In [ ]:
#Evaluating a New Neural Network

In [ ]:
mri_dataset = MRI()
mri_dataset.normalize()
model = CNN()

dataloader = DataLoader(mri_dataset, batch_size=32, shuffle=False)

In [ ]:
model.eval()
dataloader = DataLoader(mri_dataset, batch_size=32, shuffle=False)
outputs=[]
y_true = []
with torch.no_grad():
    for D in dataloader:
        image =  D['image']
        label = D['label']
        
        y_hat = model(image)
        
        outputs.append(y_hat.cpu().detach().numpy())
        y_true.append(label.cpu().detach().numpy())

In [ ]:
outputs = np.concatenate( outputs, axis=0 ).squeeze()
y_true = np.concatenate( y_true, axis=0 ).squeeze()

In [ ]:
def threshold(scores,threshold=0.50, minimum=0, maximum = 1.0):
    x = np.array(list(scores))
    x[x >= threshold] = maximum
    x[x < threshold] = minimum
    return x

In [ ]:
#Training the model

In [ ]:
eta = 0.0001
EPOCH = 400
optimizer = torch.optim.Adam(model.parameters(), lr=eta)
dataloader = DataLoader(mri_dataset, batch_size=32, shuffle=True)
model.train()

In [ ]:
for epoch in range(1, EPOCH):
    losses = []
    for D in dataloader:
        optimizer.zero_grad()
        data = D['image']
        label = D['label']
        y_hat = model(data)
        error = nn.BCELoss() 
        loss = torch.sum(error(y_hat.squeeze(), label))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

In [ ]:
#Evaluating and testing accuracy

In [ ]:
model.eval()
dataloader = DataLoader(mri_dataset, batch_size=32, shuffle=False)
outputs=[]
y_true = []
with torch.no_grad():
    for D in dataloader:
        image =  D['image']
        label = D['label']
        
        y_hat = model(image)
        
        outputs.append(y_hat.cpu().detach().numpy())
        y_true.append(label.cpu().detach().numpy())
        
outputs = np.concatenate( outputs, axis=0 )
y_true = np.concatenate( y_true, axis=0 )

In [ ]:
accuracy_score(y_true, threshold(outputs))

In [ ]:
#confusion matrix

In [ ]:
cm = confusion_matrix(y_true, threshold(outputs))
plt.figure(figsize=(16,9))

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);

ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['malignant','Benign'])
ax.yaxis.set_ticklabels(['malignant','Benign'])

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(outputs)
plt.axvline(x=len(malignant), color='r', linestyle='--')
plt.grid()

In [ ]:
#Visualizing MRI Images with accurate results

In [ ]:
def plot_all(benign, malignant, cols=5):
    def plot_category(images, title):
        num_images = len(images)
        rows = (num_images + cols - 1) // cols

        plt.figure(figsize=(16, 2 * rows)) 
        for i, img in enumerate(images, 1):
            plt.subplot(rows, cols, i)
            plt.imshow(img)
            plt.title(title)
            plt.axis('off')
        plt.tight_layout()
        plt.show()

    # Plot benign images
    plot_category(benign, "Safe")

    # Plot malignant images
    plot_category(malignant, "Tumor")

In [ ]:
plot_all(benign, malignant)